# <div align="center"> ![Logo](https://www.ups.edu.ec/ups_portal-theme/images/ups/home/logo-ups-home.png)
# <div align="center">  kNN Classification of members of congress using similarity algorithms in Neo4j
## Materia:
### <div align="center"> Sistemas Expertos 
## Docente:
### <div align="center"> Ing. Diego Quisis
## Estudiante
### <div align="center"> Ricardo Vinicio Jara Jara

## <div align="center">KNN
### k-NN es un tipo de aprendizaje basado en instancias,o aprendizaje diferido,donde la función solo se aproxima localmente y todo el cálculo se aplaza hasta la evaluación de la función. Tanto para la clasificación como para la regresión, una técnica útil puede ser asignar pesos a las contribuciones de los vecinos, de modo que los vecinos más cercanos contribuyan más a la media que a los más distantes. Por ejemplo, un esquema de ponderación común consiste en dar a cada vecino un peso de 1/d, donde d es la distancia al vecino. Los vecinos se toman de un conjunto de objetos para los que se conoce la clase (para la clasificación k-NN) o el valor de propiedad del objeto (para la regresión k-NN). Esto se puede considerar como el conjunto de entrenamiento para el algoritmo, aunque no se requiere ningún paso de entrenamiento explícito.
    
![grafo](https://i1.wp.com/3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2014/09/k-Nearest-Neighbors-algorithm.png?resize=816%2C9999&ssl=1)   
## <div align="center"> Creacion de Nodos 


In [ ]:
LOAD CSV FROM "http://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data" as row
CREATE (p:Person)
SET p.class = row[0],
    p.features = row[1..];


![nodos](https://raw.githubusercontent.com/RicardoVinicioJara/IMG/master/votacios1.png) 

- Contar Resultados [2]


In [ ]:
MATCH (n:Person) 
WHERE "?" in n.features
RETURN count(n)


-  Votos perdidos [1]

In [ ]:
MATCH (p:Person)
WHERE '?' in p.features
WITH p,apoc.coll.occurrences(p.features,'?') as missing
RETURN missing,count(*) as times ORDER BY missing ASC

![nodos](https://raw.githubusercontent.com/RicardoVinicioJara/IMG/master/votaciones2.png) 

- Excluiremos de nuestro análisis posterior a los que no votaron.

In [ ]:
MATCH (p:Person)
WITH p,apoc.coll.occurrences(p.features,'?') as missing
WHERE missing > 6
DELETE p

### Hay tres valores posibles en los conjuntos de características. Los mapearemos de la siguiente manera:

- "y" a 1
- "n" a 0
- "?" a 0,5

#### Transformar a vector de entidades

In [ ]:
MATCH (n:Person)
UNWIND n.features as feature
WITH n,collect(CASE feature WHEN 'y' THEN 1
                            WHEN 'n' THEN 0
                            ELSE 0.5 END) as feature_vector
SET n.feature_vector = feature_vector

![nodos](https://raw.githubusercontent.com/RicardoVinicioJara/IMG/master/votaciones3.png) 

# Algoritmo clasificador kNN

In [ ]:
MATCH (test:Test)
WITH test,test.feature_vector as feature_vector
CALL apoc.cypher.run('MATCH (training:Training)
    WITH training, gds.alpha.similarity.euclideanDistance($feature_vector, training.feature_vector) AS similarity
    ORDER BY similarity ASC LIMIT 3
    RETURN collect(training.class) as classes',
    {feature_vector:feature_vector}) YIELD value
WITH test.class as class, apoc.coll.sortMaps(apoc.coll.frequencies(value.classes), '^count')[-1].item as predicted_class
WITH sum(CASE when class = predicted_class THEN 1 ELSE 0 END) as correct_predictions, count(*) as total_predictions
RETURN correct_predictions,total_predictions, correct_predictions / toFloat(total_predictions) as ratio

![nodos](https://raw.githubusercontent.com/RicardoVinicioJara/IMG/master/votaciones5.png) 

## Conclusiones
Este método visto anteriormente no ayuda para hacer una clasificación binaria, ya que el algoritmo toma la mayoría de las clases de vecinos y los puede ir puntuando de acuerdo con si similitud. 
